# 네이버 금융에서 주식 시세 데이터 수집하기

https://finance.naver.com/item/main.naver?code=005930

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

URL_NAVERFIN_SISE = 'https://finance.naver.com/item/sise_day.naver?code={code}&page={page}'

def _get_diffsigns(res, columnidx_diff=2):
    soup = BeautifulSoup(res.text, 'lxml')
    rows = soup.findAll('tr')
    list_diffsign = []
    if len(rows) > 3:
        for row in rows[1::]:
            cols = row.findAll('td')
            if len(cols) < 3:
                continue
            col = cols[columnidx_diff]
            if not col:
                continue
            img = col.find('img')
            if img:
                if img.attrs.get('alt') == '상승':
                    list_diffsign.append(1)
                elif img.attrs.get('alt') == '하락':
                    list_diffsign.append(-1)
            else:
                list_diffsign.append(0)
    return list_diffsign

def get_stockchart(code, pages=1):
    data = []
    date_last = ''
    cnt = 0

    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'cache-control': 'no-cache',
        'pragma': 'no-cache',
        'sec-ch-ua': '"Whale";v="3", " Not;A Brand";v="99", "Chromium";v="96"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.76 Whale/3.12.129.34 Safari/537.36',
    }

    while True:
        res = requests.get(URL_NAVERFIN_SISE.format(code=code, page=cnt+1), headers=headers)
        if res.status_code != 200:
            break
        if res.text == '':
            break
        list_df = pd.read_html(res.text)
        if len(list_df) < 2:
            break
        df = list_df[0]
        df = df.dropna()
        df = df.rename(columns={
            '날짜': 'date', '종가': 'close', '전일비': 'diff', 
            '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'
        })
        df['date'] = df['date'].apply(lambda x: x.replace('.', ''))
        if len(df) == 0:
            break
        if date_last == df.iloc[-1]['date']:
            break
        date_last = df.iloc[-1]['date']

        list_diffsign = _get_diffsigns(res)
        
        list_item = df.to_dict(orient='records')
        for item, diffsign in zip(list_item, list_diffsign):
            item['diff'] = item['diff'] * diffsign
        data.extend(list_item)

        cnt += 1
        if pages and (cnt >= pages):
            break
    return data

get_stockchart('005930')

[{'date': '20220106',
  'close': 77100.0,
  'diff': -300.0,
  'open': 76700.0,
  'high': 77600.0,
  'low': 76600.0,
  'volume': 9023144.0},
 {'date': '20220105',
  'close': 77400.0,
  'diff': -1300.0,
  'open': 78800.0,
  'high': 79000.0,
  'low': 76400.0,
  'volume': 25470640.0},
 {'date': '20220104',
  'close': 78700.0,
  'diff': 100.0,
  'open': 78800.0,
  'high': 79200.0,
  'low': 78300.0,
  'volume': 12427416.0},
 {'date': '20220103',
  'close': 78600.0,
  'diff': 300.0,
  'open': 79400.0,
  'high': 79800.0,
  'low': 78200.0,
  'volume': 13502112.0},
 {'date': '20211230',
  'close': 78300.0,
  'diff': -500.0,
  'open': 78900.0,
  'high': 79500.0,
  'low': 78100.0,
  'volume': 14236700.0},
 {'date': '20211229',
  'close': 78800.0,
  'diff': -1500.0,
  'open': 80200.0,
  'high': 80200.0,
  'low': 78500.0,
  'volume': 19794795.0},
 {'date': '20211228',
  'close': 80300.0,
  'diff': 100.0,
  'open': 80200.0,
  'high': 80400.0,
  'low': 79700.0,
  'volume': 18226325.0},
 {'date': '2021